In [1]:
!pip install japanize-matplotlib

In [2]:
import os
import zipfile
from glob import glob
from pathlib import Path
import pdb
import random
from tqdm import tqdm

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib inline
import seaborn as sns
plt.style.use('fivethirtyeight')

import warnings
warnings.simplefilter('ignore')

from contextlib import contextmanager
from time import time

class Timer:
    """処理時間を表示するクラス
    with Timer(prefix=f'pred cv={i}'):
        y_pred_i = predict(model, loader=test_loader)
    
    with Timer(prefix='fit fold={} '.format(i)):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)
    """
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' '):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

#最大表示列数の指定
pd.set_option('display.max_columns', 50)
#最大表示行数の指定
pd.set_option('display.max_rows', 50)

def seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
seed(42)

In [3]:
data_path = Path('../data')

In [4]:
# データの生成
# 動物の種類
dataType = 2
df = pd.read_csv(os.path.join(data_path, 'decisionTree/animals.csv'))

X_train = df.drop('Y',axis=1)
y_train = df[['Y']].values

In [5]:
X_train

,食性x1,発生形態x2,体温x3
0,肉食,卵生,恒温
1,肉食,胎生,恒温
2,草食,胎生,恒温
3,肉食,卵生,変温
4,草食,卵生,恒温


In [6]:
y_train

array([['鳥類'],
       ['哺乳類'],
       ['哺乳類'],
       ['爬虫類'],
       ['鳥類']], dtype=object)

In [7]:
class decisionTree:
    """決定木モデル"""
    # スペースの記号
    space="　　　"
    lineFlag = False

    def __init__(self,X,Y,version=2):
        """
        args:
            x: 
                入力データ（データ数×次元数のnumpy.ndarray）
            y:
                出力データ（データ数×1のnumpy.ndarray）
            version:
                決定木のバージョン番号（1: ID3,2: CART）
        """
        self.X = X
        self.Y = Y
        self.version = version
        
        # 情報量を計算する関数infoFuncをversionに基づき設定
        if self.version == 1: # ID3（情報エントロピー）
            self.infoFunc = self.compEntropy
        elif self.version == 2: # CART（ジニ不純度）
            self.infoFunc = self.compGini

    def compEntropy(self,Y):
        """
        情報エントロピーの計算
        args:
            Y:
                出力データ（データ数×1のnumpy.ndarray）
        """
        probs = [np.sum(Y==y)/len(Y) for y in np.unique(Y)]

        return -np.sum(probs*np.log2(probs))

    def compGini(self,Y):
        """
        ジニ不純度の計算
        args:
            Y:
                出力データ（データ数×1のnumpy.ndarray）
        """
        probs = [np.sum(Y==y)/len(Y) for y in np.unique(Y)]

        return 1 - np.sum(np.square(probs))
    
    def selectX(self,X,Y):
        """
        説明変数の選択
        args:
            X: 
                入力データ（データ数×説明変数の数のdataframe）
            Y:
                出力データ（データ数×1のnumpy.ndarray）
        """
        # 出力Yの情報エントロピーまたはジニ不純度の計算
        allInfo = self.infoFunc(Y)

        # 各説明変数の平均情報量および利得の記録
        colInfos = []
        gains = []

        # 説明変数のループ
        for col in X.columns:
            
            # 説明変数を限定した平均情報エントロピーまたはジニ不純度の計算
            colInfo = np.sum([np.sum(X[col]==value)/len(X)*
                self.infoFunc(Y[X[col]==value]) for value in np.unique(X[col])])
            colInfos.append(colInfo)

            # 利得の計算およgainsに記録
            gains.append(allInfo-colInfo)

        # 最大利得を返す
        return np.argmax(gains),allInfo

    def delCol(self,X,Y,col,value):
        """
        説明変数の削除
        args:
            X: 
                入力データ（データ数×カラム数のdataframe）
            Y:
                出力データ（データ数×1のnumpy.ndarray）
            col: 
                削除する説明変数の名前
            value: 
                説明変数の値
        """
        # 説明変数colの削除
        subX = X[X[col]==value]
        subX = subX.drop(col,axis=1)

        # 目的変数から値を削除
        subY = Y[X[col]==value]

        return subX,subY

    def train(self,X=[],Y=[],layer=0):
        """
        決定木の作成
        args:
            X: 
                入力データ（データ数×カラム数のdataframe）
            Y:
                出力データ（データ数×1のnumpy.ndarray）
            layer: 
                階層番号（整数スカラー、デフォルトでは0）
        """
        if not len(X): X = self.X
        if not len(Y): Y = self.Y

        # 葉ノードの標準出力
        if self.infoFunc(Y) == 0:
            print(f" --> {Y[0][0]}")
            return Y[0][0]
        else:
            print("\n",end="")

        # 説明変数の選択
        colInd,allInfo = self.selectX(X,Y)

        # 説明変数名の取得
        col = X.columns[colInd]

        # 説明変数colの値ごとに枝を分岐
        for value in np.unique(X[col]):
        
            # 説明変数colの削除
            subX,subY = self.delCol(X,Y,col,value)

            #-----------
            # 分岐ノードの標準出力
            if self.lineFlag:
                print(f"{self.space*(layer-1)}｜")
            self.lineFlag = True

            if layer > 0:
                print(f"{self.space*(layer-1)}＋― ",end="")

            print(f"{col} ({round(allInfo,2)}) = '{value}' ({round(self.infoFunc(subY),2)})",end="")
            #-----------

            # 分岐先の枝で決定木を作成
            self.train(subX,subY,layer+1)

In [8]:
# 決定木の作成（version=1: ID3,version=2: CART）
myModel = decisionTree(X_train, y_train,version=2)
myModel.train()


発生形態x2 (0.64) = '卵生' (0.44)
｜
＋― 体温x3 (0.44) = '変温' (0.0) --> 爬虫類
｜
＋― 体温x3 (0.44) = '恒温' (0.0) --> 鳥類
｜
発生形態x2 (0.64) = '胎生' (0.0) --> 哺乳類


In [10]:
# データの生成
# テニスをするか否か
dataType = 1
df = pd.read_csv(os.path.join(data_path, 'decisionTree/playTennis.csv'))

X_train = df.drop('Y',axis=1)
y_train = df[['Y']].values

In [11]:
X_train

,天気x1,気温x2,風x3
0,晴れ,暑い,弱い
1,晴れ,暑い,強い
2,晴れ,普通,弱い
3,晴れ,普通,強い
4,曇り,暑い,弱い
5,曇り,普通,弱い
6,曇り,普通,強い
7,雨,普通,弱い
8,雨,普通,強い


In [12]:
y_train

array([['No'],
       ['No'],
       ['Yes'],
       ['No'],
       ['No'],
       ['Yes'],
       ['No'],
       ['No'],
       ['No']], dtype=object)

In [13]:
# 決定木の作成（version=1: ID3,version=2: CART）
myModel = decisionTree(X_train, y_train,version=1)
myModel.train()


風x3 (0.76) = '弱い' (0.97)
｜
＋― 気温x2 (0.97) = '普通' (0.92)
　　　｜
　　　＋― 天気x1 (0.92) = '晴れ' (-0.0) --> Yes
　　　｜
　　　＋― 天気x1 (0.92) = '曇り' (-0.0) --> Yes
　　　｜
　　　＋― 天気x1 (0.92) = '雨' (-0.0) --> No
｜
＋― 気温x2 (0.97) = '暑い' (-0.0) --> No
｜
風x3 (0.76) = '強い' (-0.0) --> No


In [14]:
# 決定木の作成（version=1: ID3,version=2: CART）
myModel = decisionTree(X_train, y_train,version=2)
myModel.train()


風x3 (0.35) = '弱い' (0.48)
｜
＋― 気温x2 (0.48) = '普通' (0.44)
　　　｜
　　　＋― 天気x1 (0.44) = '晴れ' (0.0) --> Yes
　　　｜
　　　＋― 天気x1 (0.44) = '曇り' (0.0) --> Yes
　　　｜
　　　＋― 天気x1 (0.44) = '雨' (0.0) --> No
｜
＋― 気温x2 (0.48) = '暑い' (0.0) --> No
｜
風x3 (0.35) = '強い' (0.0) --> No
